In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import statsmodels.api as sma 


from itertools import combinations
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from importlib import reload
from sklearn.metrics import roc_auc_score
from numpy.linalg import norm
from sklearn.decomposition import PCA

import sys
sys.path.append('/home/guest/Документы/src/cxs')
import parenclitic_oop_parallel as prct
import parenclitic_oop as znn

# Gorban poited to L1-norm, L2-norm, graph strength, PCA broken stick as chars
# no solution for broken stick yet ...
def gorban_processing(G, var_exp=0.95):

    strength_sum = sum(dict(G.degree(weight='weight')).values())
    A = nx.to_numpy_array(G)
    L1 = norm(A, 1)
    L2 = norm(A, 2)
    pca = PCA(n_components=var_exp)
    dim = pca.fit_transform(A).shape[1]
    return np.array([L1, L2, strength_sum, dim])


In [4]:
for name in ['ideal', 'broken', 'noisy']:

    # loading data ...
    data = pd.read_csv('/home/guest/Документы/src/cxs/test/' + name + '_sphere_10_65_65_20_20.csv')
    data = data.drop('Unnamed: 0', axis=1)

    X_train = data[data['label'] == 'TRAIN'].set_index('id').drop(['label', 'score'], axis=1)
    X_test = data[data['label'] == 'TEST'].set_index('id').drop(['label', 'score'], axis=1)
    y_train = data[data['label'] == 'TRAIN'].set_index('id')['score']
    y_test = data[data['label'] == 'TEST'].set_index('id')['score']

    nodes_lst = X_train.columns

    X_healthy = data[(data['label'] == 'TRAIN') & 
        (data['score'] == 0)].set_index('id').drop(['label', 'score'], axis=1)

    print(name + ' loaded')

    # creating graph objects ...
    clf = prct.Zanin(X_healthy, nodes_lst)

    clf.fit(
        X_train.append(X_test), # full table of ideal sphere
        X_train.index.append(X_test.index) # full index of ideal sphere
        )

    # make table of chars for ideal sphere ...
    chars = pd.DataFrame([[k] + [f for f in gorban_processing(v,0.99)] for k,v in clf.graphs.items()])
    chars = chars.set_index(chars.iloc[:,0]).drop(chars.iloc[:,0].name, axis=1)

    # new train/test data with gorban chars 
    train_data = chars.loc[X_train.index,:]
    test_data = chars.loc[X_test.index,:]

    # train and test with classifier
    lrcv_clf = LogisticRegressionCV(cv=5, random_state=0).fit(train_data, y_train)
    auc_roc = roc_auc_score(y_test, lrcv_clf.predict_proba(test_data)[:, 1])

    print('\n')
    print(f'### ROC AUC for {str.capitalize(name)} is {auc_roc:.5f} ###')
    print('\n')

    

ideal loaded
Data for healthy loaded successfully
Models fitted successfully
Zanin dict of graphs is ready ...


### ROC AUC for Ideal is 1.00000 ###


broken loaded
Data for healthy loaded successfully
Models fitted successfully
Zanin dict of graphs is ready ...


### ROC AUC for Broken is 0.98750 ###


noisy loaded
Data for healthy loaded successfully
Models fitted successfully
Zanin dict of graphs is ready ...


### ROC AUC for Noisy is 0.77750 ###




## Adjust for PCA ...

In [163]:
from sklearn.datasets import load_breast_cancer

breast = load_breast_cancer()
breast_data = breast.data
breast_labels = breast.target
labels = np.reshape(breast_labels,(569,1))
final_breast_data = np.concatenate([breast_data,labels],axis=1)

breast_dataset = pd.DataFrame(final_breast_data)
features = breast.feature_names
features_labels = np.append(features,'label')
breast_dataset.columns = features_labels
breast_dataset['label'].replace(0, 'Benign',inplace=True)
breast_dataset['label'].replace(1, 'Malignant',inplace=True)

x = breast_dataset.loc[:, features].values
x = StandardScaler().fit_transform(x)

feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
normalised_breast = pd.DataFrame(x,columns=feat_cols)

normalised_breast.tail()


feature0  feature1  feature2  feature3  feature4  feature5  feature6  \
564  2.110995  0.721473  2.060786  2.343856  1.041842  0.219060  1.947285   
565  1.704854  2.085134  1.615931  1.723842  0.102458 -0.017833  0.693043   
566  0.702284  2.045574  0.672676  0.577953 -0.840484 -0.038680  0.046588   
567  1.838341  2.336457  1.982524  1.735218  1.525767  3.272144  3.296944   
568 -1.808401  1.221792 -1.814389 -1.347789 -3.112085 -1.150752 -1.114873   

     feature7  feature8  feature9  ...  feature20  feature21  feature22  \
564  2.320965 -0.312589 -0.931027  ...   1.901185   0.117700   1.752563   
565  1.263669 -0.217664 -1.058611  ...   1.536720   2.047399   1.421940   
566  0.105777 -0.809117 -0.895587  ...   0.561361   1.374854   0.579001   
567  2.658866  2.137194  1.043695  ...   1.961239   2.237926   2.303601   
568 -1.261820 -0.820070 -0.561032  ...  -1.410893   0.764190  -1.432735   

     feature23  feature24  feature25  feature26  feature27  feature28  \
564   2.015301   0.378365  -0.273318   0.664512   1.629151  -1.360158   
565   1.494959  -0.691230  -0.394820   0.236573   0.733827  -0.531855   
566   0.427906  -0.809587   0.350735   0.326767   0.414069  -1.104549   
567   1.653171   1.430427   3.904848   3.197605   2.289985   1.919083   
568  -1.075813  -1.859019  -1.207552  -1.305831  -1.745063  -0.048138   

     feature29  
564  -0.709091  
565  -0.973978  
566  -0.318409  
567   2.219635  
568  -0.751207  

[5 rows x 30 columns]

In [167]:
pca_breast = PCA(n_components=2)
principalComponents_breast = pca_breast.fit_transform(x)

principal_breast_Df = pd.DataFrame(data = principalComponents_breast
             , columns = ['principal component 1', 'principal component 2'])

principal_breast_Df.tail()   
pca_breast.explained_variance_ratio_ 

array([0.44272026, 0.18971182])

In [2]:
data = pd.read_csv('/home/guest/Документы/src/cxs/test/ideal_sphere_10_65_65_20_20.csv')
data = data.drop('Unnamed: 0', axis=1)

X_train = data[data['label'] == 'TRAIN'].set_index('id').drop(['label', 'score'], axis=1)
X_test = data[data['label'] == 'TEST'].set_index('id').drop(['label', 'score'], axis=1)
y_train = data[data['label'] == 'TRAIN'].set_index('id')['score']
y_test = data[data['label'] == 'TEST'].set_index('id')['score']

nodes_lst = X_train.columns

X_healthy = data[(data['label'] == 'TRAIN') & 
    (data['score'] == 0)].set_index('id').drop(['label', 'score'], axis=1)

In [4]:
# 1
healthy = znn.DataFrameLoader(X_healthy, nodes_lst)

# 2
features = znn.Features(healthy)
features.fit()

# 3
models = znn.Model(healthy, features)
models.fit()

# 4
train_data = znn.NewData(   
    X_train, 
    healthy, 
    features, 
    models)

# 5
train_data.fit()

# 6
graphs = znn.GraphBasket(train_data, features)
graphs.fit()

graphs.graphs

{1: <networkx.classes.graph.Graph at 0x7f5565d3aac0>,
 2: <networkx.classes.graph.Graph at 0x7f55689557f0>,
 3: <networkx.classes.graph.Graph at 0x7f55689551c0>,
 4: <networkx.classes.graph.Graph at 0x7f5568955190>,
 5: <networkx.classes.graph.Graph at 0x7f556894d340>,
 6: <networkx.classes.graph.Graph at 0x7f556894d3d0>,
 7: <networkx.classes.graph.Graph at 0x7f556894d2e0>,
 8: <networkx.classes.graph.Graph at 0x7f556894d370>,
 9: <networkx.classes.graph.Graph at 0x7f556894d100>,
 10: <networkx.classes.graph.Graph at 0x7f556894d3a0>,
 11: <networkx.classes.graph.Graph at 0x7f556894d430>,
 12: <networkx.classes.graph.Graph at 0x7f556894d5b0>,
 13: <networkx.classes.graph.Graph at 0x7f556894d460>,
 14: <networkx.classes.graph.Graph at 0x7f556894d640>,
 15: <networkx.classes.graph.Graph at 0x7f556894d670>,
 16: <networkx.classes.graph.Graph at 0x7f556894d6d0>,
 17: <networkx.classes.graph.Graph at 0x7f556894d2b0>,
 18: <networkx.classes.graph.Graph at 0x7f556894d6a0>,
 19: <networkx.clas